This notebook is was used to figure out how to solve the portfolio optimization problem using a LP/MIP/QP solver. This notebook contains a prototype that attempts to use [CVXPY](https://www.cvxpy.org) to solve the problem.

The problem requires support for:
* Quadratic programming (QP) - supports minimizing a quadratic objective function (i.e., sum of the squared difference)
* Mixed-integer programming - supports integer variables (i.e., number of funds)
* Linear constraints - support constraining overall allocation to 100% and the allocation to any specific asset class or fund to be less than 100%

Because of these requirements, the solver needs to support mixed-integer quadratic programming (MIQP).

The CVXPY library supports MIQP when the SCIP solver is used.

In [ ]:
# import required packaages
import pandas as pd
import cvxpy as cp
import numpy as np

In [ ]:
# load the data
from collections import defaultdict
default_types = defaultdict(lambda: float, Ticker="str")
file_path = "../data/exposure_matrix.csv"
data = pd.read_csv(file_path, dtype=default_types)
data

In [ ]:
# Extract fund_matrix (all rows except the footer and first column)
fund_matrix = data.iloc[:-1, 1:].values
fund_matrix

In [ ]:
# Extract desired_allocations (footer row, excluding the first column)
desired_allocations = data.iloc[-1, 1:].values
desired_allocations

In [ ]:
# Extract fund tickers (first column, excluding the footer row)
fund_tickers = data.iloc[:-1, 0].values
fund_tickers

In [ ]:
# Extract asset classes (header row, excluding the first column)
asset_classes = data.columns[1:]
asset_classes

In [ ]:
# Define the optimization problem
num_funds = fund_matrix.shape[0]
x = cp.Variable(num_funds)  # Allocation to each fund
z = cp.Variable(num_funds, boolean=True)  # Binary selection variables

# Resulting portfolio allocation
portfolio_allocation = fund_matrix.T @ x

sparsity_weight = 0.01
max_funds = 7

# Objective: Minimize the squared difference between actual and desired allocations
objective = cp.Minimize(
    cp.sum_squares(portfolio_allocation - desired_allocations)
    + sparsity_weight * cp.sum(z) # Penalize the number of funds
)

# Constraints
constraints = [
    cp.sum(x) == 1,  # Allocations must sum to 100%
    x >= 0,          # No negative allocation
    x <= 1,          # Maximum allocation per fund
    x <= z,          # Link x and z (if z=0, x=0)
    cp.sum(z) <= max_funds,  # Number of funds used is <= max_funds
]

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()
print(f"Solver status: {problem.status}")

In [ ]:
# Output results
print("Optimal Fund Allocations:")
print(f"{"Ticker":10}{"Allocation":>10}")
print(f"{"========":<10}{"==========":>10}")
for ticker, allocation in zip(fund_tickers, x.value):
    print(f"{ticker:<10}{allocation:10.2%}")

print(type(portfolio_allocation.value))

print("\nResulting Asset Class Allocations:")
print(f"{"Asset Class":20}{"Actual":>10}{"Target":>10}{"Diff":>10}")
for asset_class, actual, target in zip(asset_classes, portfolio_allocation.value, desired_allocations):
    diff = actual - target
    print(f"{asset_class:20}{actual:10.2%}{target:10.2%}{diff:10.2%}")

print("\nObjective Value (total deviation):", problem.value)